# Geospatial Analysis of Schools in Peru

## 👩‍💻 Task 1: Static Maps by School Level

In [ ]:
pip install geopandas pandas matplotlib openpyxl folium

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
import branca
import os

In [ ]:
# Cargar la base de colegios
escuelas = pd.read_excel("listado_iiee.xlsx")
escuelas

In [ ]:
# Cargar el shapefile de distritos de Perú 
districts = gpd.read_file('Shapefile/DISTRITOS.shp')
districts

In [ ]:
# Crear GeoDataFrame de puntos: Definir sistema de coordenadas y crear puntos a partir de las coordenadas (lon y lat)
schools_geo = gpd.GeoDataFrame(
    escuelas, 
    crs="EPSG:4326",
    geometry=gpd.points_from_xy(escuelas['Longitud'], escuelas['Latitud'])
)

In [ ]:
# Intersección colegios - distritos
schools_geo = gpd.overlay(schools_geo, districts, how='intersection')
schools_geo

# Antes: 114919 rows
# Ahora: 114893 rows

In [ ]:
# Visualización rápida
fig, ax = plt.subplots(figsize=(12,10))
districts.boundary.plot(ax=ax, color='black')
schools_geo.plot(ax=ax, color='red', markersize=5)
plt.show()

### (Paréntesis) <br>
Antes de la intersección: 114919 rows <br>
Después de la intersección: 114893 rows <br>
¿Qué pasó?

In [ ]:
# Pre-intersección: guarda una copia original del GeoDataFrame
schools_geo_original = gpd.GeoDataFrame(
    escuelas, 
    crs="EPSG:4326",
    geometry=gpd.points_from_xy(escuelas['Longitud'], escuelas['Latitud'])
)

In [ ]:
# Después del overlay: compara los identificadores
# 2.1 Colegios que sobrevivieron (después del overlay)
colegios_despues = set(schools_geo['Código Modular'])

# 2.2 Colegios originales
colegios_antes = set(schools_geo_original['Código Modular'])

# 2.3 Colegios perdidos
colegios_perdidos = colegios_antes - colegios_despues

print(f"Total colegios perdidos: {len(colegios_perdidos)}")

In [ ]:
# Ver detalle de los colegios perdidos
colegios_perdidos_df = schools_geo_original[
    schools_geo_original['Código Modular'].isin(colegios_perdidos)
]

# Mostrar algunas columnas relevantes
colegios_perdidos_df[['Código Modular', 'Nombre de SS.EE.', 'Departamento', 'Provincia', 'Distrito', 'Latitud', 'Longitud']].head(26)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Crear un GeoDataFrame de colegios perdidos
perdidos_gdf = gpd.GeoDataFrame(
    colegios_perdidos_df,
    crs="EPSG:4326",
    geometry=gpd.points_from_xy(colegios_perdidos_df['Longitud'], colegios_perdidos_df['Latitud'])
)

# Plot
fig, ax = plt.subplots(figsize=(10,10))
districts.boundary.plot(ax=ax, edgecolor='black')
perdidos_gdf.plot(ax=ax, color='red', markersize=20, label='Colegios Perdidos')
plt.legend()
plt.title("Ubicación de colegios perdidos tras overlay")
plt.axis('off')
plt.show()

Diagnóstico visual: El gráfico sugiere que los 26 colegios fueron omitidos por encontrarse en los límites. Por lo tanto, al no tratarse de un error en la intersección, procederemos con la muestra de 114,893 colegios.
### (Fin del Paréntesis)

### Continuación: Creación de mapas estáticos que presenten la distribución de colegios por distrito para cada nivel educativo

In [ ]:
# Trabajar variables

# Crea una nueva columna llamada Nivel a partir de la columna original Nivel / Modalidad, pero: Convierte todo el texto a minúsculas (lower()).
schools_geo['Nivel'] = schools_geo['Nivel / Modalidad'].str.lower()

Tabla del número de colegios por nivel educativo

In [ ]:
import pandas as pd

# Asegurarse de que el nivel esté en minúsculas
schools_geo['Nivel'] = schools_geo['Nivel'].str.lower()

# Definir los niveles que vamos a desagregar
niveles = ['inicial', 'primaria', 'secundaria']

# Calcular directamente en la base original
total_inicial = len(schools_geo[schools_geo['Nivel'].str.contains('inicial', case=False, na=False)])
total_primaria = len(schools_geo[schools_geo['Nivel'].str.contains('primaria', case=False, na=False)])
total_secundaria = len(schools_geo[schools_geo['Nivel'].str.contains('secundaria', case=False, na=False)])
total_general = total_inicial + total_primaria + total_secundaria

# Crear la tabla resumen
tabla_totales = pd.DataFrame({
    'Nivel Educativo': ['Inicial', 'Primaria', 'Secundaria', 'Total General'],
    'Número de Colegios': [total_inicial, total_primaria, total_secundaria, total_general]
})

# Dar formato con separadores de miles
tabla_totales['Número de Colegios'] = tabla_totales['Número de Colegios'].apply(lambda x: f"{x:,}")

# Mostrar la tabla
tabla_totales

Colegios excluídos (no contienen la palabra inicial, primaria, ni secundaria)

In [ ]:
# Definir niveles válidos
niveles_validos = ['inicial', 'primaria', 'secundaria']

# Detectar colegios que NO tienen ninguno de los niveles válidos
colegios_excluidos = schools_geo[
    ~schools_geo['Nivel'].str.contains('|'.join(niveles_validos), case=False, na=False)
]

# Mostrar cuántos colegios excluidos hay
print(f"Colegios excluidos del análisis: {len(colegios_excluidos)}")

# Mostrar los primeros ejemplos
colegios_excluidos[['Código Modular', 'Nombre de SS.EE.', 'Nivel / Modalidad']].head(20)

Hasta este punto, hemos encontrado que: <br>
Total de colegios Inicial: 54,822 <br>
Total de colegios Primaria: 39,266 <br>
Total de colegios Secundaria: 15,786 <br>
Colegios excluidos del análisis: 5019 <br>
Total Inicial-Primaria-Secundaria + Excluídos: 114,893. 

Dado que los colegios excluídos representan una cantidad no considerable, procedemos con el análisis considerando solo aquellos colegios que son de nivel inicial, primaria o secundaria (explícitamente). 

Tabla del número de colegios por distrito

In [ ]:
# Asegurarse que el Ubigeo esté bien formateado
schools_geo['Ubigeo'] = schools_geo['Ubigeo'].astype(str).str.zfill(6)

# 1. Contar número de colegios por distrito
conteo_colegios_distrito = schools_geo.groupby('Ubigeo').size().reset_index(name='Total_Colegios')

# 2. Calcular la suma total de colegios en el país
suma_total_colegios = conteo_colegios_distrito['Total_Colegios'].sum()

# 3. Mostrar resultados
print("Primeros distritos con su cantidad de colegios:")
print(conteo_colegios_distrito.head(10))

print("\nSuma total de colegios en el país:")
print(f"{suma_total_colegios:,}")  # separador de miles

Mapas estáticos que presenten la distribución de colegios por distrito para cada nivel educativo

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Asegurarte de que los IDs estén correctos
schools_geo['Ubigeo'] = schools_geo['Ubigeo'].astype(str).str.zfill(6)
districts['IDDIST'] = districts['IDDIST'].astype(str).str.zfill(6)
schools_geo['Nivel'] = schools_geo['Nivel'].str.lower()

# Definimos los niveles
niveles = ['inicial', 'primaria', 'secundaria']

# Para cada nivel: contar, sumar y mapear
for nivel in niveles:
    # 1. Filtrar colegios del nivel actual
    filtered = schools_geo[schools_geo['Nivel'].str.contains(nivel, case=False, na=False)]

    # 2. Contar número de colegios por distrito
    conteo = filtered.groupby('Ubigeo').size().reset_index(name='Total_Colegios')

    # 3. Sumar total de colegios de ese nivel
    suma_total = conteo['Total_Colegios'].sum()
    print(f"\nNivel: {nivel.capitalize()}")
    print(f"Total nacional de colegios {nivel.capitalize()}: {suma_total:,} colegios")

    # Asegurar formato de string con ceros a la izquierda
    districts['IDDIST'] = districts['IDDIST'].astype(str).str.zfill(6)
    conteo['Ubigeo'] = conteo['Ubigeo'].astype(str).str.zfill(6)

    # 4. Merge shapefile distritos + conteo
    districts_plot = districts.merge(conteo, left_on='IDDIST', right_on='Ubigeo', how='left')
    districts_plot['Total_Colegios'] = districts_plot['Total_Colegios'].fillna(0)

    # 5. Plot
    fig, ax = plt.subplots(figsize=(14, 12))
    districts_plot.plot(
        column='Total_Colegios',
        cmap='YlOrRd',
        linewidth=0.5,
        edgecolor='black',
        legend=True,
        ax=ax
    )

    ax.set_title(f'Distribución de colegios - Nivel {nivel.capitalize()} (Total: {suma_total:,})', fontsize=18)
    ax.axis('off')
    plt.tight_layout()
    plt.show()

Zoom

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Asegurarse que IDs estén bien formateados
schools_geo['Ubigeo'] = schools_geo['Ubigeo'].astype(str).str.zfill(6)
districts['IDDIST'] = districts['IDDIST'].astype(str).str.zfill(6)
schools_geo['Nivel'] = schools_geo['Nivel'].str.lower()

# Definimos los niveles
niveles = ['inicial', 'primaria', 'secundaria']

# Creamos un diccionario para guardar las tablas por nivel
tablas_nivel = {}

# Para cada nivel: contar, sumar y generar tabla
for nivel in niveles:
    # 1. Filtrar colegios del nivel actual
    filtered = schools_geo[schools_geo['Nivel'].str.contains(nivel, case=False, na=False)]

    # 2. Contar número de colegios por distrito
    conteo = filtered.groupby('Ubigeo').size().reset_index(name='Total_Colegios')

    # 3. Merge shapefile distritos + conteo
    tabla_distritos = districts.merge(conteo, left_on='IDDIST', right_on='Ubigeo', how='left')

    # 4. Llenar distritos sin colegios con 0
    tabla_distritos['Total_Colegios'] = tabla_distritos['Total_Colegios'].fillna(0).astype(int)

    # 5. Mantener columnas relevantes
    tabla_distritos = tabla_distritos[['DEPARTAMEN', 'PROVINCIA', 'DISTRITO', 'IDDIST', 'Total_Colegios']]

    # 6. Guardar tabla en el diccionario
    tablas_nivel[nivel] = tabla_distritos

    # 7. Mostrar resumen en consola
    suma_total = tabla_distritos['Total_Colegios'].sum()
    print(f"Nivel: {nivel.capitalize()} | Total nacional de colegios: {suma_total:,} colegios")

# --- Ahora tienes 3 tablas almacenadas en `tablas_nivel` ---

# Por ejemplo:
# tabla inicial
print("\nTabla de colegios de Inicial:")
print(tablas_nivel['inicial'].head())

# tabla primaria
print("\nTabla de colegios de Primaria:")
print(tablas_nivel['primaria'].head())

# tabla secundaria
print("\nTabla de colegios de Secundaria:")
print(tablas_nivel['secundaria'].head())

## Task 2: Proximity Analysis


In [ ]:
# Filtrar las regiones de interés: Huancavelica y Huancayo
filtered_schools = schools_geo[
    (schools_geo['Departamento'].str.contains('HUANCAVELICA', case=False, na=False)) |
    (schools_geo['Departamento'].str.contains('HUANCAYO', case=False, na=False))
]
filtered_schools

In [ ]:
# Separar primarias y secundarias
primarias = filtered_schools[filtered_schools['Nivel'].str.contains('primaria', case=False, na=False)].copy()
secundarias = filtered_schools[filtered_schools['Nivel'].str.contains('secundaria', case=False, na=False)].copy()

In [ ]:
# Cambiar la proyección a metros
primarias = primarias.to_crs(epsg=32718)
secundarias = secundarias.to_crs(epsg=32718)

¿Qué es .to_crs(epsg=32718)? <br>
crs significa Coordinate Reference System (Sistema de Referencia de Coordenadas). <br>
El sistema de referencia original de schools_geo es EPSG:4326, que usa grados (latitud y longitud). Pero para medir distancias reales (por ejemplo, hacer un círculo de 5 km), los grados no sirven directamente. Entonces, cambias la proyección a un sistema en metros, usando: <br>
EPSG:32718 ➔ UTM Zona 18S, usado para Perú. <br>
👉 EPSG:32718 convierte tus puntos a un sistema de coordenadas donde 1 unidad = 1 metro.

In [ ]:
# Crear un buffer de 5 km alrededor de cada primaria
primarias['buffer_5km'] = primarias.geometry.buffer(5000)  # 5 km = 5000 metros

In [ ]:
# Contar cuántas secundarias caen dentro de cada buffer
conteo_secundarias = []

for idx, primaria in primarias.iterrows():
    buffer = primaria['buffer_5km']
    # Buscar secundarias dentro del buffer
    secundarias_cercanas = secundarias[secundarias.geometry.within(buffer)]
    conteo_secundarias.append(len(secundarias_cercanas))

# Agregar el conteo como nueva columna
primarias['secundarias_cerca'] = conteo_secundarias

# Ahora cada primaria tiene cuántas secundarias tiene cerca en un radio de 5 km.

In [ ]:
# Identificar la primaria con más y menos secundarias cercanas
# Primaria con menos secundarias cerca
primaria_min = primarias.sort_values('secundarias_cerca').iloc[0]

# Primaria con más secundarias cerca
primaria_max = primarias.sort_values('secundarias_cerca', ascending=False).iloc[0]

In [ ]:
# Plot para primaria con MENOS secundarias
fig, ax = plt.subplots(figsize=(10, 10))
secundarias.plot(ax=ax, color='lightgray', markersize=5, label='Secundarias')
gpd.GeoSeries(primaria_min.geometry).plot(ax=ax, color='blue', markersize=50, label='Primaria (mínimo)')
gpd.GeoSeries(primaria_min.buffer_5km).plot(ax=ax, color='none', edgecolor='blue', linestyle='--', linewidth=2)
secundarias_within_buffer = secundarias[secundarias.geometry.within(primaria_min.buffer_5km)]
secundarias_within_buffer.plot(ax=ax, color='red', markersize=20, label='Secundarias dentro')
plt.title(f'Primaria con MENOS secundarias cerca ({primaria_min["secundarias_cerca"]} secundarias)', fontsize=14)
plt.legend()
plt.axis('off')
plt.show()

# Plot para primaria con MÁS secundarias
fig, ax = plt.subplots(figsize=(10, 10))
secundarias.plot(ax=ax, color='lightgray', markersize=5, label='Secundarias')
gpd.GeoSeries(primaria_max.geometry).plot(ax=ax, color='green', markersize=50, label='Primaria (máximo)')
gpd.GeoSeries(primaria_max.buffer_5km).plot(ax=ax, color='none', edgecolor='green', linestyle='--', linewidth=2)
secundarias_within_buffer_max = secundarias[secundarias.geometry.within(primaria_max.buffer_5km)]
secundarias_within_buffer_max.plot(ax=ax, color='red', markersize=20, label='Secundarias dentro')
plt.title(f'Primaria con MÁS secundarias cerca ({primaria_max["secundarias_cerca"]} secundarias)', fontsize=14)
plt.legend()
plt.axis('off')
plt.show()

# Task 3: Interactive Mapping with Folium


### Choropleth con Folium por niveles

In [ ]:
import folium
from folium import Choropleth, LayerControl

# Asegurar IDs
schools_geo['Ubigeo'] = schools_geo['Ubigeo'].astype(str).str.zfill(6)
districts['IDDIST'] = districts['IDDIST'].astype(str).str.zfill(6)
schools_geo['Nivel'] = schools_geo['Nivel'].str.lower()

# Conteos por nivel
niveles = ['inicial', 'primaria', 'secundaria']
conteos = {}

for nivel in niveles:
    filtered = schools_geo[schools_geo['Nivel'].str.contains(nivel, case=False, na=False)]
    conteo = filtered.groupby('Ubigeo').size().reset_index(name='Total_Colegios')
    conteos[nivel] = conteo

# Mapa base
m = folium.Map(location=[-12.04318, -77.02824], zoom_start=6, tiles='OpenStreetMap')  # Perú centro

# Agregar cada capa de choropleth
for nivel, conteo in conteos.items():
    choropleth = Choropleth(
        geo_data=districts.to_json(),
        name=f"{nivel.capitalize()}",
        data=conteo,
        columns=['Ubigeo', 'Total_Colegios'],
        key_on='feature.properties.IDDIST',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f"Colegios de {nivel.capitalize()}"
    )
    choropleth.add_to(m)

# Control de capas
LayerControl(collapsed=False).add_to(m)

# Mostrar mapa
m

# High School Proximity Visualization

In [ ]:
# Identificación de primaria_min y primaria_max, pero esta vez considerando todos los departamentos

import geopandas as gpd

# 1. Filtrar colegios de tipo primaria y secundaria de TODO EL PAÍS
primarias = schools_geo[schools_geo['Nivel'].str.contains('primaria', case=False, na=False)].copy()
secundarias = schools_geo[schools_geo['Nivel'].str.contains('secundaria', case=False, na=False)].copy()

# 2. Cambiar proyección a metros (para cálculo de distancias reales)
primarias = primarias.to_crs(epsg=32718)    # EPSG:32718 = UTM zona 18 Sur (ideal para Perú)
secundarias = secundarias.to_crs(epsg=32718)

# 3. Crear buffer de 5 km alrededor de cada primaria
primarias['buffer_5km'] = primarias.geometry.buffer(5000)  # Radio de 5 km

# 4. Contar cuántas secundarias caen dentro del buffer de cada primaria
conteo_secundarias = []

for idx, primaria in primarias.iterrows():
    buffer = primaria['buffer_5km']
    # Buscar secundarias que estén DENTRO del círculo
    secundarias_cercanas = secundarias[secundarias.geometry.within(buffer)]
    conteo_secundarias.append(len(secundarias_cercanas))

# 5. Agregar el conteo como nueva columna
primarias['secundarias_cerca'] = conteo_secundarias

# 6. Definir primaria con más y menos secundarias cercanas
primaria_min = primarias.sort_values('secundarias_cerca').iloc[0]
primaria_max = primarias.sort_values('secundarias_cerca', ascending=False).iloc[0]

# 7. Imprimir resultados rápidos
print(f"Primaria con menos secundarias cerca: {primaria_min['Nombre de SS.EE.']} ({primaria_min['secundarias_cerca']} secundarias)")
print(f"Primaria con más secundarias cerca: {primaria_max['Nombre de SS.EE.']} ({primaria_max['secundarias_cerca']} secundarias)")

In [ ]:
import matplotlib.pyplot as plt

# Nueva función: usar coordenadas reales
def plot_proximity_real(nombre_primaria, secundaria_coords, titulo):
    fig, ax = plt.subplots(figsize=(8, 8))

    # 1. Dibujar el círculo de 5 km
    circle = plt.Circle((0, 0), 5, color='royalblue', alpha=0.7)
    ax.add_patch(circle)

    # 2. Dibujar la escuela primaria en el centro
    ax.scatter(0, 0, color='black', s=400)
    ax.text(0, 0, nombre_primaria, color='white', fontsize=12, ha='center', va='center')

    # 3. Dibujar las secundarias con sus coordenadas reales relativas
    for x, y in secundaria_coords:
        ax.scatter(x, y, color='black', s=400)
        ax.text(x, y, "Secundaria", color='white', fontsize=8, ha='center', va='center')

    # 4. Línea de referencia de 5 km
    ax.plot([0, 5], [0, 0], color='black')
    ax.text(2.5, 0.2, "5 km", fontsize=10, ha='center', va='bottom')

    # Configuración final
    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)
    ax.set_aspect('equal')
    ax.axis('off')
    plt.title(titulo, fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
# Seleccionar las secundarias cercanas
secundarias_within_buffer = secundarias[secundarias.geometry.within(primaria_min['buffer_5km'])]
secundarias_within_buffer_max = secundarias[secundarias.geometry.within(primaria_max['buffer_5km'])]

In [ ]:
secundarias_coords_min = [
    ((geom.x - primaria_min.geometry.x) / 1000, (geom.y - primaria_min.geometry.y) / 1000)
    for geom in secundarias_within_buffer.geometry
]

secundarias_coords_max = [
    ((geom.x - primaria_max.geometry.x) / 1000, (geom.y - primaria_max.geometry.y) / 1000)
    for geom in secundarias_within_buffer_max.geometry
]

In [ ]:
# Para la primaria con MENOS secundarias cerca
plot_proximity_real(
    nombre_primaria="Primaria",
    secundaria_coords=secundarias_coords_min,
    titulo=f"Primaria con Menos Secundarias ({len(secundarias_coords_min)} escuelas)"
)

# Para la primaria con MÁS secundarias cerca
plot_proximity_real(
    nombre_primaria="Primaria",
    secundaria_coords=secundarias_coords_max,
    titulo=f"Primaria con Más Secundarias ({len(secundarias_coords_max)} escuelas)"
)

### Analysis

In [ ]:
primaria_min

La escuela primaria con menos secundarias cerca está ubicada en el Departamento de Ucayali, Provicina de Purus y Distrito de Purus. 

In [ ]:
primaria_max

La escuela primaria con más secundarias cerca está ubicada en el Departamento de Lima, Provicina de Lima y Distrito de Los Olivos. 

### Análisis Comparativo



### 📍 Comparación entre el Distrito de Los Olivos y el Distrito de Purús

### 🏙️ Distrito: Los Olivos

##### 🌎 Terreno
- **Superficie total:** 18.25 km²
- **Altitud media:** 75 m s. n. m.
- **Notas:** Distrito urbano consolidado en Lima Norte.
- **Fuente:** [Wikipedia - Los Olivos](https://es.wikipedia.org/wiki/Distrito_de_Los_Olivos)

##### 🚗 Accesibilidad
- **Acceso:** Alto
- **Vías principales:** 
  - Panamericana Norte
  - Av. Canta Callao
  - Av. Naranjal
- **Notas:** Conexiones estratégicas con otros distritos de Lima Norte.
- **Fuente:** [Municipalidad de Los Olivos - Plan de Riesgos](https://sigrid.cenepred.gob.pe/sigridv3/storage/biblioteca/8028_plan-de-prevencion-y-reduccion-del-riesgo-de-desastres-por-sismos-del-distrito-de-los-olivos-2019-2022.pdf)

##### 🏢 Características Urbanas vs. Rurales
- **Zona:** Urbana
- **Notas:** Considerado el distrito socioeconómicamente mejor consolidado de Lima Norte.
- **Fuente:** [Wikipedia - Los Olivos](https://es.wikipedia.org/wiki/Distrito_de_Los_Olivos)

---

### 🌳 Distrito: Purús

##### 🌎 Terreno
- **Superficie total:** 17,848 km²
- **Altitud media:** 232 m s. n. m.
- **Notas:** 
  - Parte de la Amazonía peruana.
  - Habitado por etnias pano (grupos sharanahua: "Onicoin" y "Yura").
  - Limita con Brasil, Madre de Dios y Atalaya.
- **Fuente:** [Wikipedia - Purús](https://es.wikipedia.org/wiki/Distrito_de_Pur%C3%BAs)

##### 🚤 Accesibilidad
- **Acceso:** Muy limitado
- **Vías principales:** Solo acceso aéreo o fluvial.
- **Fuente:** [Municipalidad Provincial de Purús](https://sinia.minam.gob.pe/sites/default/files/sinia/archivos/public/docs/mp_purus_ucayali_documento_pt_2022.pdf)

##### 🏡 Características Urbanas vs. Rurales
- **Zona:** Rural
- **Notas:** Predominancia de características rurales y baja densidad poblacional.
- **Fuente:** [INEI - Censos Nacionales](https://www.inei.gob.pe/media/MenuRecursivo/publicaciones_digitales/Est/Lib1554/25TOMO_01.pdf)

---


## Streamlit

In [ ]:
pip install streamlit

In [ ]:
pip install streamlit-folium

### Revisar Doc App.py 

In [ ]:
'''
# app.py

# --- Librerías necesarias ---
import streamlit as st
import geopandas as gpd
import pandas as pd
import folium
from folium import Choropleth, LayerControl
import matplotlib.pyplot as plt
from streamlit_folium import folium_static

# --- Cargar los datos ---
# Asegúrate de tener los archivos en las rutas correctas:
# listado_iiee.xlsx y Shapefile/DISTRITOS.shp

# 1. Cargar colegios
escuelas = pd.read_excel('listado_iiee.xlsx')

# 2. Crear GeoDataFrame de colegios
schools_geo = gpd.GeoDataFrame(
    escuelas,
    crs="EPSG:4326",
    geometry=gpd.points_from_xy(escuelas['Longitud'], escuelas['Latitud'])
)

# 3. Cargar shapefile de distritos
districts = gpd.read_file('Shapefile/DISTRITOS.shp')

# --- Preprocesar districts para Streamlit ---

# 1. QUEDARTE SOLO CON ID Y GEOMETRÍA
districts = districts[['IDDIST', 'geometry']].copy()

# 2. SIMPLIFICAR POLÍGONOS
districts['geometry'] = districts['geometry'].simplify(tolerance=0.01, preserve_topology=True)

# 4. Ajustes de formato
schools_geo['Ubigeo'] = schools_geo['Ubigeo'].astype(str).str.zfill(6)
districts['IDDIST'] = districts['IDDIST'].astype(str).str.zfill(6)
schools_geo['Nivel'] = schools_geo['Nivel / Modalidad'].str.lower()

# --- Configurar página de Streamlit ---
st.set_page_config(page_title="Análisis Geoespacial de Colegios en Perú", layout="wide")

# --- Tabs ---
tab1, tab2, tab3 = st.tabs(["🗂️ Descripción de Datos", "🗺️ Mapas Estáticos", "🌍 Mapas Dinámicos"])

# ============================================================
# Tab 1: Descripción de datos
# ============================================================

with tab1:
    st.header("🗂️ Descripción de Datos")

    st.subheader("Unidad de Análisis")
    st.write("""
    La unidad de análisis de este proyecto es cada institución educativa (colegio) en el Perú.
    Cada colegio está identificado de manera única por sus coordenadas geográficas (latitud y longitud) y su nivel educativo (Inicial, Primaria o Secundaria).
    El análisis espacial agrega los colegios a nivel distrital para observar patrones de acceso educativo en el ámbito local.
    """)

    st.subheader("Fuentes de Datos")
    st.write("""
    - **Base de datos de colegios**: Ministerio de Educación del Perú (MINEDU), extraído del portal SIGMED (https://sigmed.minedu.gob.pe/mapaeducativo/).
    - **Shapefile de límites distritales**: Fuente oficial del Instituto Nacional de Estadística e Informática (INEI).
    """)

    st.subheader("Supuestos y Preprocesamiento")
    st.write("""
    - Solo se incluyeron colegios que cuentan con coordenadas geográficas válidas y completas.
    - Se categorizaron los colegios en tres niveles educativos: Inicial, Primaria y Secundaria.
    - Se excluyeron las instituciones sin clasificación de nivel o con errores de geolocalización.
    - La agregación distrital asume que los límites provistos por el INEI son actuales y precisos.
    - El análisis de proximidad basado en radio utiliza un buffer de 5 km alrededor de las escuelas primarias.
    """)

# ============================================================
# Tab 2: Mapas Estáticos
# ============================================================

with tab2:
    st.header("🗺️ Mapas Estáticos")

    niveles = ['inicial', 'primaria', 'secundaria']

    for nivel in niveles:
        st.subheader(f"Distribución de colegios - Nivel {nivel.capitalize()}")

        # Filtrar colegios del nivel
        filtered = schools_geo[schools_geo['Nivel'].str.contains(nivel, case=False, na=False)]

        # Contar por distrito
        conteo = filtered.groupby('Ubigeo').size().reset_index(name='Total_Colegios')

        # Merge shapefile
        districts_plot = districts.merge(conteo, left_on='IDDIST', right_on='Ubigeo', how='left')
        districts_plot['Total_Colegios'] = districts_plot['Total_Colegios'].fillna(0)

        # Plot
        fig, ax = plt.subplots(figsize=(10, 10))
        districts_plot.plot(
            column='Total_Colegios',
            cmap='YlOrRd',
            linewidth=0.5,
            edgecolor='black',
            legend=True,
            ax=ax
        )
        ax.set_title(f'Distribución de colegios - Nivel {nivel.capitalize()}', fontsize=16)
        ax.axis('off')
        st.pyplot(fig)

# ============================================================
# Tab 3: Mapas Dinámicos
# ============================================================

with tab3:
    st.header("🌍 Mapas Dinámicos")

    st.subheader("Distribución de colegios por nivel educativo (mapa interactivo)")

    niveles = ['inicial', 'primaria', 'secundaria']
    conteos = {}

    for nivel in niveles:
        filtered = schools_geo[schools_geo['Nivel'].str.contains(nivel, case=False, na=False)]
        conteo = filtered.groupby('Ubigeo').size().reset_index(name='Total_Colegios')
        conteos[nivel] = conteo

    # Crear el mapa base
    m = folium.Map(location=[-12.04318, -77.02824], zoom_start=6, tiles='OpenStreetMap')

    # Agregar capas
    for nivel, conteo in conteos.items():
        Choropleth(
            geo_data=districts.to_json(),
            data=conteo,
            columns=['Ubigeo', 'Total_Colegios'],
            key_on='feature.properties.IDDIST',
            fill_color='YlOrRd',
            fill_opacity=0.7,
            line_opacity=0.2,
            name=f"{nivel.capitalize()}",
            legend_name=f"Colegios de {nivel.capitalize()}"
        ).add_to(m)

    LayerControl(collapsed=False).add_to(m)

    folium_static(m, width=1200, height=700)
'''

### Análisis extra

### 1. Gráfico de barras horizontales: Top 10 distritos con más colegios

In [ ]:
top10 = conteo_colegios_distrito.sort_values('Total_Colegios', ascending=False).head(10)
top10.plot(kind='barh', x='Ubigeo', y='Total_Colegios', color='skyblue', figsize=(10, 6))
plt.xlabel('Número de colegios')
plt.title('Top 10 distritos con más colegios')
plt.gca().invert_yaxis()  # Para que el distrito con más colegios esté arriba
plt.tight_layout()
plt.show()

In [ ]:
# Ubigeos que quieres identificar
ubigeos_interes = ['150132', '150135', '150103', '150110', '150142', '150143', '070101', '150117', '150133', '150125']

# Buscar sus nombres en la base de distritos
distritos_interes = districts[districts['IDDIST'].isin(ubigeos_interes)][['IDDIST', 'DISTRITO', 'PROVINCIA', 'DEPARTAMEN']]

# Mostrar tabla
print(distritos_interes)

### 2. Histograma: Distribución del número de colegios por distrito

In [ ]:
conteo_colegios_distrito['Total_Colegios'].plot(
    kind='hist', bins=30, color='orchid', edgecolor='black', figsize=(10, 6)
)
plt.xlabel('Número de colegios por distrito')
plt.title('Distribución de colegios por distrito')
plt.tight_layout()
plt.show()

In [ ]:
# cd /Users/rominarattoyanez/hw3
# streamlit run app.py

In [ ]:
# pip list

## Zoom en Ica

In [ ]:
# Buscar colegios que contengan "cruz" en el nombre
colegios_cruz = secundarias_ica[secundarias_ica['Nombre de SS.EE.'].str.contains('cruz', case=False, na=False)]

# Mostrar resultados
colegios_cruz

In [ ]:
# Filtrar solo Ica
primarias_ica = schools_geo[
    (schools_geo['Nivel'].str.contains("primaria", case=False, na=False)) &
    (schools_geo['Departamento'].str.upper() == 'ICA')
].copy()

secundarias_ica = schools_geo[
    (schools_geo['Nivel'].str.contains("secundaria", case=False, na=False)) &
    (schools_geo['Departamento'].str.upper() == 'ICA')
].copy()

# Crear geometría
primarias_ica = primarias_ica.set_geometry(gpd.points_from_xy(primarias_ica.Longitud, primarias_ica.Latitud)).set_crs(epsg=4326).to_crs(epsg=32718)
secundarias_ica = secundarias_ica.set_geometry(gpd.points_from_xy(secundarias_ica.Longitud, secundarias_ica.Latitud)).set_crs(epsg=4326).to_crs(epsg=32718)

# Crear buffer de 5km
primarias_ica['buffer_5km'] = primarias_ica.geometry.buffer(5000)

# Contar secundarias dentro del buffer
primarias_ica['conteo_secundarias'] = primarias_ica['buffer_5km'].apply(
    lambda buf: secundarias_ica[secundarias_ica.geometry.within(buf)].shape[0]
)

# Identificar primaria con más y menos secundarias cercanas
primaria_max = primarias_ica.loc[primarias_ica['conteo_secundarias'].idxmax()]
primaria_min = primarias_ica.loc[primarias_ica['conteo_secundarias'].idxmin()]

# Volver a lat/lon
primarias_ica = primarias_ica.to_crs(epsg=4326)
secundarias_ica = secundarias_ica.to_crs(epsg=4326)


In [ ]:
primaria_max

In [ ]:
primaria_min

In [ ]:
import folium
from shapely.geometry import Point
import geopandas as gpd

# Filtrar solo Ica
primarias_ica = schools_geo[
    (schools_geo['Nivel'].str.contains("primaria", case=False, na=False)) &
    (schools_geo['Departamento'].str.upper() == 'ICA')
].copy()

secundarias_ica = schools_geo[
    (schools_geo['Nivel'].str.contains("secundaria", case=False, na=False)) &
    (schools_geo['Departamento'].str.upper() == 'ICA')
].copy()

# Crear geometría
primarias_ica = primarias_ica.set_geometry(gpd.points_from_xy(primarias_ica.Longitud, primarias_ica.Latitud)).set_crs(epsg=4326).to_crs(epsg=32718)
secundarias_ica = secundarias_ica.set_geometry(gpd.points_from_xy(secundarias_ica.Longitud, secundarias_ica.Latitud)).set_crs(epsg=4326).to_crs(epsg=32718)

# Crear buffer de 5km
primarias_ica['buffer_5km'] = primarias_ica.geometry.buffer(5000)

# Contar secundarias dentro del buffer
primarias_ica['conteo_secundarias'] = primarias_ica['buffer_5km'].apply(
    lambda buf: secundarias_ica[secundarias_ica.geometry.within(buf)].shape[0]
)

# Identificar primaria con más y menos secundarias cercanas
primaria_max = primarias_ica.loc[primarias_ica['conteo_secundarias'].idxmax()]
primaria_min = primarias_ica.loc[primarias_ica['conteo_secundarias'].idxmin()]

# Volver a lat/lon
primarias_ica = primarias_ica.to_crs(epsg=4326)
secundarias_ica = secundarias_ica.to_crs(epsg=4326)

# Crear mapa centrado en Ica
m = folium.Map(location=[-14.07, -75.73], zoom_start=8, tiles='CartoDB positron')

# Agregar buffers como círculos
for _, row in primarias_ica.iterrows():
    folium.Circle(
        location=[row.geometry.y, row.geometry.x],
        radius=5000,
        color='red',
        fill=True,
        fill_opacity=0.2
    ).add_to(m)

center_lat=-14.08831
center_lon=-75.75128

# Círculo grande visual para "recorte" (fondo)
folium.Circle(
    location=[center_lat, center_lon],
    radius=100000,  # 100 km de radio
    color=None,
    fill=True,
    fill_opacity=0.07
).add_to(m)

# Marcador para mi colegio
folium.Marker(
    location=[-14.08831, -75.75128],
    popup=folium.Popup("<b>IE: DE LA CRUZ</b><br>Distrito: ICA<br>Nivel: Secundaria<br>Gestión: Privada", max_width=250),
    icon=folium.Icon(color='blue', icon='graduation-cap', prefix='fa')
).add_to(m)

# Marcador para la primaria con más secundarias
folium.Marker(
    location=[-14.07166, -75.72748],
    popup=folium.Popup(
        f"<b>Escuela con MÁS secundarias cercanas:</b><br>"
        f"{primaria_max['Código Modular']}<br>Conteo: {primaria_max['conteo_secundarias']}", max_width=250
    ),
    icon=folium.Icon(color="green", icon="arrow-up", prefix='fa')
).add_to(m)

# Marcador para la primaria con menos secundarias
folium.Marker(
    location=[-13.87274, -76.00486],
    popup=folium.Popup(
        f"<b>Escuela con MENOS secundarias cercanas:</b><br>"
        f"{primaria_min['Código Modular']}<br>Conteo: {primaria_min['conteo_secundarias']}", max_width=250
    ),
    icon=folium.Icon(color="purple", icon="arrow-down", prefix='fa')
).add_to(m)

# Mostrar mapa
m

## End